In [54]:
!pip freeze

Active code page: 65001
absl-py==0.7.1
alabaster==0.7.12
arabic-reshaper==2.0.15
asn1crypto==0.24.0
astor==0.7.1
atomicwrites==1.2.1
attrs==18.2.0
aws-xray-sdk==0.95
awscli==1.16.61
Babel==2.6.0
backcall==0.1.0
beautifulsoup4==4.7.1
bleach==2.1.3
blis==0.4.1
boto==2.48.0
boto3==1.9.50
botocore==1.12.51
bz2file==0.98
CacheControl==0.11.6
cachetools==3.1.1
catalogue==1.0.0
cchardet==2.1.4
certifi==2017.11.5
cffi==1.11.2
chardet==3.0.4
Click==7.0
colorama==0.3.9
conllu==0.11
cookies==2.2.1
coverage==4.5.3
cryptography==2.4.2
cssselect==1.0.3
cycler==0.10.0
cymem==2.0.2
Cython==0.29.7
cytoolz==0.9.0.1
datefinder==0.6.2
de-core-news-sm==2.2.5
decorator==4.3.0
dill==0.2.8.2
docker==3.5.1
docker-pycreds==0.3.0
docopt==0.6.2
docutils==0.14
dyNET==2.1
ecdsa==0.13
editdistance==0.5.2
en-core-web-sm==2.2.5
entrypoints==0.2.3
flaky==3.4.0
Flask==1.0.2
Flask-Cors==3.0.6
ftfy==5.5.0
funcy==1.14
future==0.16.0
futures==3.1.1
gast==0.2.2
gensim==3.4.0
get==2019.4.13
gevent==1.3.6
google==2.0.2
google-

In [3]:
#https://towardsdatascience.com/topic-modelling-in-python-with-nltk-and-gensim-4ef03213cd21
#https://www.kaggle.com/ryuheeeei/how-to-do-data-visualization-with-text-datasets

import warnings
warnings.filterwarnings("ignore")

# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
import matplotlib.pyplot as plt
import seaborn as sns
import mglearn
from sklearn.feature_extraction.text import CountVectorizer

In [5]:
#Reading a the 2016 CCC talks, which is stored as a CSV file
File2016_df = pd.read_csv("csvs/2016.csv", delimiter='|', header=0)#notice the delimiter is not a comma, check your files first.
print('Number of titles: {:,}\n'.format(File2016_df.shape[0]))
File2016_df.sample(5)

Number of titles: 149



,year,congress,title,abstract,link,tags
9,2016,33C3,Software Defined Emissions: ...or why adding B...,A technical talk on how to reverse-engineer el...,https://events.ccc.de/congress/2016/Fahrplan/s...,empty
105,2016,33C3,Irren ist staatlich,Dieses Jahr feiert das Informationsfreiheitsge...,https://events.ccc.de/congress/2016/Fahrplan/s...,empty
68,2016,33C3,Shining some light on the Amazon Dash button,This talk will explore the hard- and software ...,https://events.ccc.de/congress/2016/Fahrplan/s...,empty
11,2016,33C3,Console hackz 2016: Ein Geheimdienst als Zeuge...,"Last year, we demonstrated Linux running on th...",https://events.ccc.de/congress/2016/Fahrplan/s...,empty
8,2016,33C3,Dieselgate – A year later: A hackz’s review of...,At 32C3 we gave an overview on the organizatio...,https://events.ccc.de/congress/2016/Fahrplan/s...,empty


In [6]:
#reading titles 

title_dict = {}

# loop through titles
for i in range(len(File2016_df)):
    if File2016_df["title"][i] in title_dict.keys():
        title_dict[File2016_df["title"][i]] += 1
    else:
        title_dict.setdefault(File2016_df["title"][i], 1)

In [7]:
#reading abstracts 

abstract_dict = {}

# classify that the article has recommends
for i in range(len(File2016_df)):
    if File2016_df["abstract"][i] in abstract_dict.keys():
        abstract_dict[File2016_df["abstract"][i]] += 1
    else:
        abstract_dict.setdefault(File2016_df["abstract"][i], 1)

abstract_dict = [x for x in abstract_dict if str(x) != 'nan']#some talks have no abstracts

In [27]:
#spacy tokenizer

import spacy
spacy.load('en')
from spacy.lang.en import English
parser = English()
def tokenize(text):
    all_tokens = []
    tokens = parser(text)
    for token in tokens:
        if token.orth_.isspace():
            continue
        elif token.like_url:
            all_tokens.append('URL')
        elif token.orth_.startswith('@'):
            all_tokens.append('SCREEN_NAME')
        else:
            all_tokens.append(token.lower_)
    return all_tokens

In [28]:
import nltk
nltk.download('wordnet')
from nltk.corpus import wordnet as wn
def get_lemma(word):
    lemma = wn.morphy(word)
    if lemma is None:
        return word
    else:
        return lemma
    
nltk.download('stopwords')
en_stop = set(nltk.corpus.stopwords.words('english'))
de_stop = set(nltk.corpus.stopwords.words('german'))


def preprocess_text(text):
    tokens = tokenize(text)
    tokens = [token for token in tokens if len(token) > 4]
    tokens = [token for token in tokens if token not in en_stop or token in de_stop]
    tokens = [get_lemma(token) for token in tokens]
    return tokens

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\elhaj\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\elhaj\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [40]:
title_text_data = []
print(type(title_text_data))
counter = 0
for title in title_dict:
    counter = counter + 1
    title_tokens = preprocess_text(title.replace('hackz', 'hack').replace('securityz', 'security'))
    if (counter < 11):
        print(title_tokens)
    title_text_data.append(title_tokens)

<class 'list'>
['opening', 'ceremony', 'infrastructure', 'people', 'behind', 'drone', 'killing']
['global', 'assassination', 'crypto', 'fight', 'apple']
['fight', 'encryption']
['bootstrap', 'slightly', 'security', 'laptop', 'local', 'computer', 'outside', 'jurisdiction']
['enforcement', 'planet', 'security']
['money', 'technical', 'teardown', 'pegasus', 'malware', 'trident', 'security', 'chain']
['pegasus', 'internals', 'automation', 'dependency', 'aviation', 'accident']
['stink']
['dieselgate', 'later', 'review', 'dieselgate']
['software', 'define', 'emission', 'add', 'device', 'sometimes', 'smart']


In [41]:
abstract_text_data = []
counter2 = 0
for abstract in abstract_dict:
    counter2 = counter2 + 1
    abstract_tokens = preprocess_text(abstract.replace('hackz', 'hack').replace('securityz', 'security'))
    if (counter2 < 11):
        print(abstract_tokens)
    abstract_text_data.append(abstract_tokens)

['empty']
['force', 'comms', 'bomb', 'technician', 'insight', 'invisible', 'network', 'governing', 'military', 'drone', 'quest', 'accountability']
['strong', 'communications', 'encryption', 'device', 'encryption', 'legal', 'jurisdiction', 'today', 'remain', 'widely', 'available', 'software', 'programers', 'hardware', 'producer', 'increasingly', 'pressure', 'enforcement', 'policy', 'maker', 'around', 'world', 'include', 'call', 'backdoor', 'encryption', 'product', 'lecture', 'provide', 'state', 'move', 'detailing', 'happen', 'fight', 'apple', 'bernardino', 'current', 'proposal', 'weaken', 'encryption', 'covering', 'propose', 'recently', 'enact', 'discus', 'extra', 'legal', 'pressure', 'place', 'company', 'government', 'state', 'sponsor', 'malware', 'alternative', 'addition', 'weakening', 'software', 'finally', 'presentation', 'discus', 'possible', 'realistic', 'outcome', 'prediction', 'state', 'discus', 'fight', 'future', 'allow', 'security', 'communications', 'everyone']
['head', 'sour

In [42]:
#gensim for titles
#what is a Gensim? The next important object you need to familiarize with in order to work in gensim is the Corpus (a Bag of Words). That is, it is a corpus object that contains the word id and its frequency in each document. You can think of it as gensim's equivalent of a Document-Term matrix
#The main function is doc2bow , which converts a collection of words to its bag-of-words representation: a list of (word_id, word_frequency) 2-tuples
from gensim import corpora
title_dictionary = corpora.Dictionary(title_text_data)
title_corpus = [title_dictionary.doc2bow(text) for text in title_text_data]

import pickle
pickle.dump(title_corpus, open('models/title_corpus.pkl', 'wb'))#needed to serialize gensim into a stream of byes to be stored as a file.
title_dictionary.save('models/title_dictionary.gensim')

In [43]:
#gensim for abstracts
from gensim import corpora
abstract_dictionary = corpora.Dictionary(abstract_text_data)
abstract_corpus = [abstract_dictionary.doc2bow(text) for text in abstract_text_data]

import pickle
pickle.dump(abstract_corpus, open('abstract_corpus.pkl', 'wb'))
abstract_dictionary.save('abstract_dictionary.gensim')

In [50]:
#title gensim topics
# probability distribution
# Discover abstract “topics” that occur in a collection of documents (why some parts of the data are similar)
# What are the topics they are talking about in the conference talks
import gensim
NUM_TOPICS = 5
ldamodel = gensim.models.ldamodel.LdaModel(title_corpus, num_topics = NUM_TOPICS, id2word=title_dictionary, passes=10)
ldamodel.save('models/title_model.gensim')
topics = ldamodel.print_topics(num_words=4)
for topic in topics:
    print(topic)

C:\Users\elhaj\AppData\Local\Programs\Python\Python36\lib\site-packages\gensim-3.4.0-py3.6-win-amd64.egg\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\elhaj\AppData\Local\Programs\Python\Python36\lib\site-packages\gensim-3.4.0-py3.6-win-amd64.egg\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\elhaj\AppData\Local\Programs\Python\Python36\lib\site-packages\gensim-3.4.0-py3.6-win-amd64.egg\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is depreca

C:\Users\elhaj\AppData\Local\Programs\Python\Python36\lib\site-packages\gensim-3.4.0-py3.6-win-amd64.egg\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\elhaj\AppData\Local\Programs\Python\Python36\lib\site-packages\gensim-3.4.0-py3.6-win-amd64.egg\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\elhaj\AppData\Local\Programs\Python\Python36\lib\site-packages\gensim-3.4.0-py3.6-win-amd64.egg\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is depreca

C:\Users\elhaj\AppData\Local\Programs\Python\Python36\lib\site-packages\gensim-3.4.0-py3.6-win-amd64.egg\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\elhaj\AppData\Local\Programs\Python\Python36\lib\site-packages\gensim-3.4.0-py3.6-win-amd64.egg\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\elhaj\AppData\Local\Programs\Python\Python36\lib\site-packages\gensim-3.4.0-py3.6-win-amd64.egg\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is depreca

C:\Users\elhaj\AppData\Local\Programs\Python\Python36\lib\site-packages\gensim-3.4.0-py3.6-win-amd64.egg\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\elhaj\AppData\Local\Programs\Python\Python36\lib\site-packages\gensim-3.4.0-py3.6-win-amd64.egg\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\elhaj\AppData\Local\Programs\Python\Python36\lib\site-packages\gensim-3.4.0-py3.6-win-amd64.egg\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is depreca

C:\Users\elhaj\AppData\Local\Programs\Python\Python36\lib\site-packages\gensim-3.4.0-py3.6-win-amd64.egg\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\elhaj\AppData\Local\Programs\Python\Python36\lib\site-packages\gensim-3.4.0-py3.6-win-amd64.egg\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\elhaj\AppData\Local\Programs\Python\Python36\lib\site-packages\gensim-3.4.0-py3.6-win-amd64.egg\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is depreca

...

C:\Users\elhaj\AppData\Local\Programs\Python\Python36\lib\site-packages\gensim-3.4.0-py3.6-win-amd64.egg\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\elhaj\AppData\Local\Programs\Python\Python36\lib\site-packages\gensim-3.4.0-py3.6-win-amd64.egg\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\elhaj\AppData\Local\Programs\Python\Python36\lib\site-packages\gensim-3.4.0-py3.6-win-amd64.egg\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is depreca

C:\Users\elhaj\AppData\Local\Programs\Python\Python36\lib\site-packages\gensim-3.4.0-py3.6-win-amd64.egg\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\elhaj\AppData\Local\Programs\Python\Python36\lib\site-packages\gensim-3.4.0-py3.6-win-amd64.egg\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\elhaj\AppData\Local\Programs\Python\Python36\lib\site-packages\gensim-3.4.0-py3.6-win-amd64.egg\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is depreca

C:\Users\elhaj\AppData\Local\Programs\Python\Python36\lib\site-packages\gensim-3.4.0-py3.6-win-amd64.egg\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\elhaj\AppData\Local\Programs\Python\Python36\lib\site-packages\gensim-3.4.0-py3.6-win-amd64.egg\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\elhaj\AppData\Local\Programs\Python\Python36\lib\site-packages\gensim-3.4.0-py3.6-win-amd64.egg\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is depreca

C:\Users\elhaj\AppData\Local\Programs\Python\Python36\lib\site-packages\gensim-3.4.0-py3.6-win-amd64.egg\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\elhaj\AppData\Local\Programs\Python\Python36\lib\site-packages\gensim-3.4.0-py3.6-win-amd64.egg\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\elhaj\AppData\Local\Programs\Python\Python36\lib\site-packages\gensim-3.4.0-py3.6-win-amd64.egg\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is depreca

(0, '0.013*"technology" + 0.010*"surveillance" + 0.007*"state" + 0.007*"behind"')
(1, '0.017*"security" + 0.013*"space" + 0.010*"privacy" + 0.010*"knowledge"')
(2, '0.010*"privacy" + 0.010*"surveillance" + 0.007*"network" + 0.007*"attackz"')
(3, '0.012*"space" + 0.008*"internet" + 0.008*"base" + 0.008*"group"')
(4, '0.017*"security" + 0.017*"talks" + 0.013*"lightning" + 0.009*"mobile"')


In [51]:
#display and plot title topics

dictionary = gensim.corpora.Dictionary.load('models/title_dictionary.gensim')
corpus = pickle.load(open('models/title_corpus.pkl', 'rb'))
lda = gensim.models.ldamodel.LdaModel.load('models/title_model.gensim')
import pyLDAvis.gensim
lda_display = pyLDAvis.gensim.prepare(lda, title_corpus, title_dictionary, sort_topics=False)
pyLDAvis.display(lda_display, template_type='notebook')